## Feature Engineering

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.decomposition import PCA
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, make_scorer, roc_auc_score
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

from sklearn.decomposition import PCA

from ydata_profiling import ProfileReport

In [ ]:
import warnings
import pandas as pd
pd.set_option('mode.chained_assignment', None)

Preparing the Data

In [4]:
# preparing our data
raw_prices = pd.read_csv("https://storage.googleapis.com/sovai-public/random/assetalloc.csv", sep=';', parse_dates=True, index_col='Dates', dayfirst=True)
df = raw_prices.sort_values(by='Dates')
df["target"] = df["SP500"].pct_change().shift(-1)
df["target"] = np.where(df["target"]>0,1,0)
df.head()

,FTSE,EuroStoxx50,SP500,Gold,French-2Y,French-5Y,French-10Y,French-30Y,US-2Y,US-5Y,US-10Y,US-30Y,Russel2000,EuroStox_Small,FTSE_Small,MSCI_EM,CRB,target
Dates,,,,,,,,,,,,,,,,,,
1989-02-01,2039.7,875.47,297.09,392.50,99.081,99.039,99.572,100.000,100.031,100.345,101.080,101.936,154.38,117.50,1636.57,133.584,286.67,0
1989-02-02,2043.4,878.08,296.84,392.00,98.898,99.117,99.278,99.692,100.000,100.314,101.017,101.905,154.94,117.69,1642.94,135.052,287.03,1
1989-02-03,2069.9,884.09,296.97,388.75,98.907,99.002,99.145,99.178,99.812,100.062,100.921,101.718,155.69,118.62,1659.11,137.134,285.63,0
1989-02-06,2044.3,885.49,296.04,388.00,98.484,98.502,98.510,97.739,99.812,100.062,100.794,101.468,155.58,118.89,1656.86,137.037,284.69,1
1989-02-07,2072.8,883.82,299.63,392.75,98.438,98.312,98.292,97.688,99.906,100.251,101.144,102.092,156.84,118.28,1662.76,136.914,284.21,0


In [5]:
df

,FTSE,EuroStoxx50,SP500,Gold,French-2Y,French-5Y,French-10Y,French-30Y,US-2Y,US-5Y,US-10Y,US-30Y,Russel2000,EuroStox_Small,FTSE_Small,MSCI_EM,CRB,target
Dates,,,,,,,,,,,,,,,,,,
1989-02-01,2039.70,875.470,297.09,392.50,99.081,99.039,99.572,100.000,100.031,100.345,101.080,101.936,154.38,117.500,1636.57,133.584,286.67,0
1989-02-02,2043.40,878.080,296.84,392.00,98.898,99.117,99.278,99.692,100.000,100.314,101.017,101.905,154.94,117.690,1642.94,135.052,287.03,1
1989-02-03,2069.90,884.090,296.97,388.75,98.907,99.002,99.145,99.178,99.812,100.062,100.921,101.718,155.69,118.620,1659.11,137.134,285.63,0
1989-02-06,2044.30,885.490,296.04,388.00,98.484,98.502,98.510,97.739,99.812,100.062,100.794,101.468,155.58,118.890,1656.86,137.037,284.69,1
1989-02-07,2072.80,883.820,299.63,392.75,98.438,98.312,98.292,97.688,99.906,100.251,101.144,102.092,156.84,118.280,1662.76,136.914,284.21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-06,7617.70,3448.490,2759.82,1254.84,90.036,149.257,211.743,306.533,122.817,143.926,143.078,164.296,1694.05,255.104,5888.00,1059.968,437.12,1
2018-07-09,7687.99,3460.442,2784.17,1261.56,90.022,149.158,211.487,303.672,122.779,143.745,142.721,163.527,1704.60,256.680,5904.19,1075.738,437.79,1
2018-07-10,7692.04,3473.311,2793.84,1254.45,90.004,149.059,211.377,303.464,122.702,143.610,142.565,163.354,1695.62,258.043,5921.59,1076.188,438.66,0


### Train Test Split

In [6]:
from sklearn.model_selection import train_test_split
y = df.pop("target")
X = df.copy()

X_train = X[X.index.astype(str)<'2018-01-01']
y_train = y[X_train.index]
X_test = X[~X.index.isin(X_train.index)]
y_test = y[X_test.index]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

### Transforming

In [7]:
raw_price_cols = X.columns

In [12]:
np.log(df['FTSE'])

Dates
1989-02-01    7.620558
1989-02-02    7.622370
1989-02-03    7.635256
1989-02-06    7.622811
1989-02-07    7.636656
                ...   
2018-07-06    8.938230
2018-07-09    8.947415
2018-07-10    8.947941
2018-07-11    8.934845
2018-07-12    8.942635
Name: FTSE, Length: 7682, dtype: float64

In [13]:
np.log1p(df['FTSE'])

Dates
1989-02-01    7.621048
1989-02-02    7.622860
1989-02-03    7.635739
1989-02-06    7.623300
1989-02-07    7.637138
                ...   
2018-07-06    8.938361
2018-07-09    8.947545
2018-07-10    8.948071
2018-07-11    8.934977
2018-07-12    8.942765
Name: FTSE, Length: 7682, dtype: float64

In [15]:
df["FTSE_returns"] = df["FTSE"].pct_change()
df["FTSE_log"].dropna(inplace=True)
df["FTSE_log_returns"] = np.log1p(df["FTSE_returns"])

In [16]:
## Transforming 1 
# Name: Z-score Normalization of returns
# Description: Performing z-score normalization on all features, i.e., all return values. Now, the mean of all columns is 0 and s.d. is 1. 

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)
X_train_s

array([[-1.89544273, -1.62650676, -1.5024032 , ..., -1.07295059,
        -1.56486927, -0.47435859],
       [-1.89300533, -1.62412986, -1.50285424, ..., -1.0671289 ,
        -1.56013613, -0.47059038],
       [-1.87554828, -1.61865661, -1.5026197 , ..., -1.05235078,
        -1.55342334, -0.48524452],
       ...,
       [ 1.78106626,  0.80931925,  2.80147691, ...,  2.80015241,
         1.69417779,  1.04443783],
       [ 1.78251553,  0.78576426,  2.81035338, ...,  2.82747869,
         1.72381147,  1.04914809],
       [ 1.82526226,  0.76723627,  2.78522142, ...,  2.83436053,
         1.73952302,  1.05155555]])

In [17]:
## Transforming 2 
# Name: Min-max Normalization of returns
# Description: Performing min-max normalization on all features, i.e., all return values. Now, the minimum and maximum values map to 0 and 1 respectively.

scaler = MinMaxScaler()
X_train_mm = scaler.fit_transform(X_train)
X_test_mm = scaler.transform(X_test)
X_train_mm

array([[0.00868791, 0.01226235, 0.00415306, ..., 0.11270409, 0.        ,
        0.21620829],
       [0.00933731, 0.01282413, 0.00404902, ..., 0.11402611, 0.00121836,
        0.21716862],
       [0.01398842, 0.01411773, 0.00410312, ..., 0.11738202, 0.0029463 ,
        0.21343399],
       ...,
       [0.9882248 , 0.58797141, 0.99686232, ..., 0.99223181, 0.83891068,
        0.6032758 ],
       [0.98861093, 0.58240417, 0.99890972, ..., 0.99843723, 0.84653868,
        0.60447622],
       [1.        , 0.57802507, 0.99311292, ..., 1.        , 0.85058299,
        0.60508976]])

In [18]:
## Transforming 3 
# Name: Robust Scaling of returns
# Description: This scaler scales numerical features by centering them around the median and normalizing them by the interquartile range (IQR). This method is less sensitive to outliers compared to standard scaling methods, making it suitable for datasets with skewed distributions or a presence of extreme values.

scaler = RobustScaler()
X_train_r = scaler.fit_transform(X_train)
X_test_r = scaler.transform(X_test)
X_train_r

array([[-1.30453617, -1.04536244, -1.11987317, ..., -0.71267805,
        -0.62539846, -0.04385493],
       [-1.30303681, -1.04392936, -1.12020484, ..., -0.7082788 ,
        -0.62292709, -0.04170401],
       [-1.29229815, -1.04062943, -1.12003237, ..., -0.69711148,
        -0.61942204, -0.05006871],
       ...,
       [ 0.95705753,  0.42324366,  2.04498809, ...,  2.21409209,
         1.07629371,  0.82308657],
       [ 0.95794904,  0.4090419 ,  2.05151541, ...,  2.23474162,
         1.09176676,  0.82577523],
       [ 0.98424457,  0.39787103,  2.03303461, ...,  2.23994199,
         1.09997045,  0.82714943]])

### Interacting

In [19]:
# Interacting 
# Name: Ratio of Gold return to 10Y treasury
# Desciption: Both gold and treasuries are safe-haven assets and discrepancy in their ratio could be a sign of some macroeconomic event.

def gold_to_yield(df):
    teny_returns = df["US-10Y"].pct_change()
    # gold_returns = df["Gold"]
    gold_returns = df["Gold"].pct_change()
    teny_returns = teny_returns.fillna(0)
    gold_returns = gold_returns.fillna(0)
    epsilon = 1e-10
    df["gold_r__div__teny_r"] = np.divide(gold_returns, (teny_returns + 1e-10))
    return df

X_train = gold_to_yield(X_train); X_test = gold_to_yield(X_test)
X_train

,FTSE,EuroStoxx50,SP500,Gold,French-2Y,French-5Y,French-10Y,French-30Y,US-2Y,US-5Y,US-10Y,US-30Y,Russel2000,EuroStox_Small,FTSE_Small,MSCI_EM,CRB,gold_r__div__teny_r
Dates,,,,,,,,,,,,,,,,,,
1989-02-01,2039.70,875.470,297.09,392.50,99.081,99.039,99.572,100.000,100.031,100.345,101.080,101.936,154.38,117.500,1636.57,133.584,286.67,0.000000
1989-02-02,2043.40,878.080,296.84,392.00,98.898,99.117,99.278,99.692,100.000,100.314,101.017,101.905,154.94,117.690,1642.94,135.052,287.03,2.043879
1989-02-03,2069.90,884.090,296.97,388.75,98.907,99.002,99.145,99.178,99.812,100.062,100.921,101.718,155.69,118.620,1659.11,137.134,285.63,8.724099
1989-02-06,2044.30,885.490,296.04,388.00,98.484,98.502,98.510,97.739,99.812,100.062,100.794,101.468,155.58,118.890,1656.86,137.037,284.69,1.533094
1989-02-07,2072.80,883.820,299.63,392.75,98.438,98.312,98.292,97.688,99.906,100.251,101.144,102.092,156.84,118.280,1662.76,136.914,284.21,3.525563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-25,7592.66,3553.389,2683.34,1274.29,91.846,149.476,209.557,293.433,124.228,147.302,148.144,170.041,1542.93,266.238,5848.24,1141.108,431.77,0.000000
2017-12-26,7592.66,3553.389,2680.50,1282.51,91.846,149.476,209.557,293.433,124.228,147.418,148.427,170.988,1544.23,266.238,5848.24,1138.632,432.65,3.376767
2017-12-27,7620.68,3550.174,2682.62,1286.08,91.793,149.605,209.935,294.529,124.282,147.651,149.136,173.287,1543.94,267.076,5874.46,1144.390,431.77,0.582739


In [20]:
## Interacting 1 

# Name: Treasury - Equity Market Return Interaction
''' Description: Feature that represents the interaction between where the govt thinks the economy will be in a 
few years (e.g., "US-2Y," "US-5Y," "US-10Y," "US-30Y") and equity market returns. If the result is positive, it suggests
that both are assets are moving in the same direction and the magnitude represents the strength of their joint
movement. 

'''
def treasury_equity(df):
    for treasury in ["US-2Y", "US-5Y", "US-10Y", "US-30Y"]:
        economy = df[treasury].pct_change()
        equity = df["SP500"].pct_change()
        economy = economy.fillna(0)
        equity = equity.fillna(0)
        df[f"{treasury}_equity_interaction"] = np.multiply(economy, equity)
    return df
        
X_train = treasury_equity(X_train); X_test = treasury_equity(X_test)
X_train

,FTSE,EuroStoxx50,SP500,Gold,French-2Y,French-5Y,French-10Y,French-30Y,US-2Y,US-5Y,...,Russel2000,EuroStox_Small,FTSE_Small,MSCI_EM,CRB,gold_r__div__teny_r,US-2Y_equity_interaction,US-5Y_equity_interaction,US-10Y_equity_interaction,US-30Y_equity_interaction
Dates,,,,,,,,,,,,,,,,,,,,,
1989-02-01,2039.70,875.470,297.09,392.50,99.081,99.039,99.572,100.000,100.031,100.345,...,154.38,117.500,1636.57,133.584,286.67,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1989-02-02,2043.40,878.080,296.84,392.00,98.898,99.117,99.278,99.692,100.000,100.314,...,154.94,117.690,1642.94,135.052,287.03,2.043879,2.607829e-07,2.599668e-07,5.244780e-07,2.559093e-07
1989-02-03,2069.90,884.090,296.97,388.75,98.907,99.002,99.145,99.178,99.812,100.062,...,155.69,118.620,1659.11,137.134,285.63,8.724099,-8.233392e-07,-1.100170e-06,-4.161958e-07,-8.036502e-07
1989-02-06,2044.30,885.490,296.04,388.00,98.484,98.502,98.510,97.739,99.812,100.062,...,155.58,118.890,1656.86,137.037,284.69,1.533094,-0.000000e+00,-0.000000e+00,3.940874e-06,7.696842e-06
1989-02-07,2072.80,883.820,299.63,392.75,98.438,98.312,98.292,97.688,99.906,100.251,...,156.84,118.280,1662.76,136.914,284.21,3.525563,1.142061e-05,2.290534e-05,4.210924e-05,7.457608e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-25,7592.66,3553.389,2683.34,1274.29,91.846,149.476,209.557,293.433,124.228,147.302,...,1542.93,266.238,5848.24,1141.108,431.77,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2017-12-26,7592.66,3553.389,2680.50,1282.51,91.846,149.476,209.557,293.433,124.228,147.418,...,1544.23,266.238,5848.24,1138.632,432.65,3.376767,-0.000000e+00,-8.334739e-07,-2.021832e-06,-5.894391e-06
2017-12-27,7620.68,3550.174,2682.62,1286.08,91.793,149.605,209.935,294.529,124.282,147.651,...,1543.94,267.076,5874.46,1144.390,431.77,0.582739,3.437909e-07,1.250044e-06,3.777925e-06,1.063392e-05


In [21]:
## Interacting 2 

# Name: Genetic Programming for Automatic Discovery of Mathematical Relations in the data
# Description: Use symbolic regression for the algorithm to learn patterns related to the label and apply them to create new features
from gplearn.genetic import SymbolicTransformer

def genetic_feat(X_train, y_train, X_test, num_gen=20, num_comp=10):
    function_set = ['add', 'sub', 'mul', 'div',
                    'sqrt', 'log', 'abs', 'neg', 'inv','tan']
    gp = SymbolicTransformer(generations=num_gen, population_size=200,
                            hall_of_fame=100, n_components=num_comp,
                            function_set=function_set,
                            parsimony_coefficient=0.0005,
                            max_samples=0.9, verbose=1,
                            random_state=0, n_jobs=6)

    gen_feats_train = gp.fit_transform(X_train, y_train)
    gen_feats_test = gp.transform(X_test)
    return gen_feats_train, gen_feats_test

gen_feats_train, gen_feats_test = genetic_feat(X_train[raw_price_cols], y_train, X_test[raw_price_cols])

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     8.31       0.00924678        3        0.0390885        0.0314409     19.78s
   1     5.38        0.0217046        5        0.0404122        0.0604821      2.45s
   2     3.51        0.0250635        3        0.0401654        0.0422327      2.26s
   3     3.00         0.027029        4        0.0418255       0.00779273      2.11s
   4     3.42        0.0285876        4        0.0478192        0.0670179      1.88s
   5     3.85        0.0269826        4        0.0505299        0.0788278      2.03s
   6     3.96        0.0253172        4        0.0453055        0.0419538      2.05s
   7     3.93        0.0262889        4        0.0468777        0.0506545      1.76s
   8     3.96        0.0259309        4         0.045955         0.044679  

In [23]:
gen_feats_train.shape

(7543, 10)

In [24]:
## Interacting 3 

# Name: Difference in Returns of Large vs Small Companies
''' Description : This feature represents the difference in returns between FTSE and FTSE SmallCap. It 
highlights the outperformance or underperformance of the larger-cap companies compared to the smaller-cap companies.
'''
def large_small_cap_difference(df):
    large_returns = df['FTSE'].pct_change().fillna(0)
    small_returns = df['FTSE_Small'].pct_change().fillna(0)
    df['FTSE_FTSE_Small_Difference'] =  np.subtract(large_returns, small_returns)
    return df

X_train = large_small_cap_difference(X_train); X_test = large_small_cap_difference(X_test)
X_train

,FTSE,EuroStoxx50,SP500,Gold,French-2Y,French-5Y,French-10Y,French-30Y,US-2Y,US-5Y,...,EuroStox_Small,FTSE_Small,MSCI_EM,CRB,gold_r__div__teny_r,US-2Y_equity_interaction,US-5Y_equity_interaction,US-10Y_equity_interaction,US-30Y_equity_interaction,FTSE_FTSE_Small_Difference
Dates,,,,,,,,,,,,,,,,,,,,,
1989-02-01,2039.70,875.470,297.09,392.50,99.081,99.039,99.572,100.000,100.031,100.345,...,117.500,1636.57,133.584,286.67,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
1989-02-02,2043.40,878.080,296.84,392.00,98.898,99.117,99.278,99.692,100.000,100.314,...,117.690,1642.94,135.052,287.03,2.043879,2.607829e-07,2.599668e-07,5.244780e-07,2.559093e-07,-0.002078
1989-02-03,2069.90,884.090,296.97,388.75,98.907,99.002,99.145,99.178,99.812,100.062,...,118.620,1659.11,137.134,285.63,8.724099,-8.233392e-07,-1.100170e-06,-4.161958e-07,-8.036502e-07,0.003126
1989-02-06,2044.30,885.490,296.04,388.00,98.484,98.502,98.510,97.739,99.812,100.062,...,118.890,1656.86,137.037,284.69,1.533094,-0.000000e+00,-0.000000e+00,3.940874e-06,7.696842e-06,-0.011012
1989-02-07,2072.80,883.820,299.63,392.75,98.438,98.312,98.292,97.688,99.906,100.251,...,118.280,1662.76,136.914,284.21,3.525563,1.142061e-05,2.290534e-05,4.210924e-05,7.457608e-05,0.010380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-25,7592.66,3553.389,2683.34,1274.29,91.846,149.476,209.557,293.433,124.228,147.302,...,266.238,5848.24,1141.108,431.77,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
2017-12-26,7592.66,3553.389,2680.50,1282.51,91.846,149.476,209.557,293.433,124.228,147.418,...,266.238,5848.24,1138.632,432.65,3.376767,-0.000000e+00,-8.334739e-07,-2.021832e-06,-5.894391e-06,0.000000
2017-12-27,7620.68,3550.174,2682.62,1286.08,91.793,149.605,209.935,294.529,124.282,147.651,...,267.076,5874.46,1144.390,431.77,0.582739,3.437909e-07,1.250044e-06,3.777925e-06,1.063392e-05,-0.000793


### Mapping

In [25]:
# Mapping 

# Name: First prinicipal component of all of the assets returns
# Description:For stocks the first component resmbles the return of the market, for multiple asset classes it could resemble a 'universal' asset class


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

def pca_first(X_train, X_test):

    X_train_returns = X_train[raw_price_cols].pct_change(axis=0).fillna(0)
    X_train_returns = X_train_returns.replace([np.inf, -np.inf], np.nan).dropna()
    # print(X_train_returns)
    X_test_returns = X_test[raw_price_cols].pct_change(axis=0).fillna(0)
    X_test_returns = X_test_returns.replace([np.inf, -np.inf], np.nan).dropna()

    sc = StandardScaler() # Normalize the data
    X_train_normalized = sc.fit_transform(X_train_returns)
    # print(X_train_normalized)
    X_test_normalized = sc.transform(X_test_returns)
    # Apply PCA and transform data
    pca = PCA(1)
    X_train['pca'] = pca.fit_transform(X_train_normalized)
    X_test['pca'] = pca.transform(X_test_normalized)
    loadings = pca.components_
    print("Feature Loadings:\n", loadings)
    return X_train, X_test

X_train_pca, X_test_pca = pca_first(X_train, X_test)
X_train_pca


Feature Loadings:
 [[ 0.27338244  0.28925149  0.23468168 -0.02298693 -0.194819   -0.23203742
  -0.2242631  -0.20875917 -0.26427817 -0.29879956 -0.30367338 -0.28862951
   0.24288332  0.29014099  0.24119046  0.23194152  0.10261159]]


,FTSE,EuroStoxx50,SP500,Gold,French-2Y,French-5Y,French-10Y,French-30Y,US-2Y,US-5Y,...,FTSE_Small,MSCI_EM,CRB,gold_r__div__teny_r,US-2Y_equity_interaction,US-5Y_equity_interaction,US-10Y_equity_interaction,US-30Y_equity_interaction,FTSE_FTSE_Small_Difference,pca
Dates,,,,,,,,,,,,,,,,,,,,,
1989-02-01,2039.70,875.470,297.09,392.50,99.081,99.039,99.572,100.000,100.031,100.345,...,1636.57,133.584,286.67,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-0.009470
1989-02-02,2043.40,878.080,296.84,392.00,98.898,99.117,99.278,99.692,100.000,100.314,...,1642.94,135.052,287.03,2.043879,2.607829e-07,2.599668e-07,5.244780e-07,2.559093e-07,-0.002078,1.373297
1989-02-03,2069.90,884.090,296.97,388.75,98.907,99.002,99.145,99.178,99.812,100.062,...,1659.11,137.134,285.63,8.724099,-8.233392e-07,-1.100170e-06,-4.161958e-07,-8.036502e-07,0.003126,2.638858
1989-02-06,2044.30,885.490,296.04,388.00,98.484,98.502,98.510,97.739,99.812,100.062,...,1656.86,137.037,284.69,1.533094,-0.000000e+00,-0.000000e+00,3.940874e-06,7.696842e-06,-0.011012,2.087348
1989-02-07,2072.80,883.820,299.63,392.75,98.438,98.312,98.292,97.688,99.906,100.251,...,1662.76,136.914,284.21,3.525563,1.142061e-05,2.290534e-05,4.210924e-05,7.457608e-05,0.010380,0.183672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-25,7592.66,3553.389,2683.34,1274.29,91.846,149.476,209.557,293.433,124.228,147.302,...,5848.24,1141.108,431.77,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-0.009163
2017-12-26,7592.66,3553.389,2680.50,1282.51,91.846,149.476,209.557,293.433,124.228,147.418,...,5848.24,1138.632,432.65,3.376767,-0.000000e+00,-8.334739e-07,-2.021832e-06,-5.894391e-06,0.000000,-0.444257
2017-12-27,7620.68,3550.174,2682.62,1286.08,91.793,149.605,209.935,294.529,124.282,147.651,...,5874.46,1144.390,431.77,0.582739,3.437909e-07,1.250044e-06,3.777925e-06,1.063392e-05,-0.000793,-0.914195


In [26]:
## Mapping 1

# Name: SVD of all of the assets returns
''' Description: Singular Value Decomposition (SVD) is a linear dimensionality reduction 
technique that is commonly used for feature extraction and noise reduction. It can be applied 
to both dense and sparse data.
'''
from sklearn.decomposition import TruncatedSVD
def svd(X_train, X_test):
    svd = TruncatedSVD(n_components=1, random_state=42)
    
    # Assuming X_train_returns and X_test_returns are your normalized returns data
    X_train_returns = X_train[raw_price_cols].pct_change(axis=0).fillna(0)
    X_train_returns = X_train_returns.replace([np.inf, -np.inf], np.nan).dropna()
    X_test_returns = X_test[raw_price_cols].pct_change(axis=0).fillna(0)
    X_test_returns = X_test_returns.replace([np.inf, -np.inf], np.nan).dropna()
    
    sc = StandardScaler()  # Normalize the data
    X_train_normalized = sc.fit_transform(X_train_returns)
    X_test_normalized = sc.transform(X_test_returns)
    
    X_train['svd'] = svd.fit_transform(X_train_normalized)
    X_test['svd'] = svd.transform(X_test_normalized)
    
    return X_train, X_test

X_train_svd, X_test_svd = svd(X_train, X_test)
X_train_svd


,FTSE,EuroStoxx50,SP500,Gold,French-2Y,French-5Y,French-10Y,French-30Y,US-2Y,US-5Y,...,MSCI_EM,CRB,gold_r__div__teny_r,US-2Y_equity_interaction,US-5Y_equity_interaction,US-10Y_equity_interaction,US-30Y_equity_interaction,FTSE_FTSE_Small_Difference,pca,svd
Dates,,,,,,,,,,,,,,,,,,,,,
1989-02-01,2039.70,875.470,297.09,392.50,99.081,99.039,99.572,100.000,100.031,100.345,...,133.584,286.67,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-0.009470,-0.009470
1989-02-02,2043.40,878.080,296.84,392.00,98.898,99.117,99.278,99.692,100.000,100.314,...,135.052,287.03,2.043879,2.607829e-07,2.599668e-07,5.244780e-07,2.559093e-07,-0.002078,1.373297,1.373297
1989-02-03,2069.90,884.090,296.97,388.75,98.907,99.002,99.145,99.178,99.812,100.062,...,137.134,285.63,8.724099,-8.233392e-07,-1.100170e-06,-4.161958e-07,-8.036502e-07,0.003126,2.638858,2.638858
1989-02-06,2044.30,885.490,296.04,388.00,98.484,98.502,98.510,97.739,99.812,100.062,...,137.037,284.69,1.533094,-0.000000e+00,-0.000000e+00,3.940874e-06,7.696842e-06,-0.011012,2.087348,2.087348
1989-02-07,2072.80,883.820,299.63,392.75,98.438,98.312,98.292,97.688,99.906,100.251,...,136.914,284.21,3.525563,1.142061e-05,2.290534e-05,4.210924e-05,7.457608e-05,0.010380,0.183672,0.183672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-25,7592.66,3553.389,2683.34,1274.29,91.846,149.476,209.557,293.433,124.228,147.302,...,1141.108,431.77,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-0.009163,-0.009163
2017-12-26,7592.66,3553.389,2680.50,1282.51,91.846,149.476,209.557,293.433,124.228,147.418,...,1138.632,432.65,3.376767,-0.000000e+00,-8.334739e-07,-2.021832e-06,-5.894391e-06,0.000000,-0.444257,-0.444257
2017-12-27,7620.68,3550.174,2682.62,1286.08,91.793,149.605,209.935,294.529,124.282,147.651,...,1144.390,431.77,0.582739,3.437909e-07,1.250044e-06,3.777925e-06,1.063392e-05,-0.000793,-0.914195,-0.914195


In [27]:
## Mapping 2 
# Name: Autoencoding returns
''' Description: Autoencoders, as unsupervised algorithms, are designed for data compression. 
Comprising an encoder, a decoder, and a loss function, they evaluate the information loss between 
the compressed and decompressed data representations.
'''

from sklearn.neural_network import MLPRegressor
def autoencoder(X_train, X_test):
    sc = StandardScaler()

    # Assuming X_train_returns and X_test_returns are your normalized returns data
    X_train_returns = X_train[raw_price_cols].pct_change(axis=0).fillna(0)
    X_train_returns = X_train_returns.replace([np.inf, -np.inf], np.nan).dropna()
    X_test_returns = X_test[raw_price_cols].pct_change(axis=0).fillna(0)
    X_test_returns = X_test_returns.replace([np.inf, -np.inf], np.nan).dropna()

    # Normalize the data
    X_train_normalized = sc.fit_transform(X_train_returns)
    X_test_normalized = sc.transform(X_test_returns)

    autoencoder = MLPRegressor(hidden_layer_sizes=(10,), activation='relu', solver='adam', max_iter=1000, random_state=42)

    # Train the autoencoder on the normalized data
    X_train_ae = autoencoder.fit(X_train_normalized, X_train_normalized).predict(X_train_normalized)
    X_test_ae = autoencoder.predict(X_test_normalized)

    return X_train_ae, X_test_ae

X_train_ae, X_test_ae = autoencoder(X_train[raw_price_cols], X_test[raw_price_cols])
X_train_ae

array([[-0.0519742 , -0.05076474, -0.03235119, ..., -0.00586673,
        -0.03555214, -0.0323303 ],
       [ 0.16967968,  0.11201396,  0.0499138 , ...,  0.54373202,
         0.89728179,  0.32234219],
       [ 0.82361593,  0.86848877,  0.04041941, ...,  1.11890686,
         1.42394746, -1.21233614],
       ...,
       [ 0.23783631,  0.15189234, -0.03730264, ...,  0.45716115,
         0.54302825, -0.52999038],
       [-0.16107344, -0.2141089 ,  0.02388801, ...,  0.28636263,
         0.84864913,  0.27607779],
       [-0.05630339, -0.11724688, -0.63209088, ...,  0.27834748,
         0.2865488 ,  0.06138441]])

In [28]:
X_train_ae.shape

(7543, 17)

In [29]:
!pip install umap-learn

In [30]:
## Mapping 3 
# Name: UMAPping returns 
''' Description: UMAP feature provides a non-linear dimensionality reduction for financial returns data, 
excelling in preserving both local and global structures with computational efficiency and user-friendly 
parameter tuning compared to t-SNE and autoencoders.
'''
from umap import UMAP
def umap(X_train, X_test):  
    X_train_returns = X_train[raw_price_cols].pct_change(axis=0).fillna(0)
    X_train_returns = X_train_returns.replace([np.inf, -np.inf], np.nan).dropna()
    X_test_returns = X_test[raw_price_cols].pct_change(axis=0).fillna(0)
    X_test_returns = X_test_returns.replace([np.inf, -np.inf], np.nan).dropna()

    # Normalize the data
    scaler = StandardScaler()
    X_train_normalized = scaler.fit_transform(X_train_returns)
    X_test_normalized = scaler.transform(X_test_returns)

    umap_model = UMAP(n_components=1)
    X_train['umap'] = umap_model.fit_transform(X_train_normalized)
    X_test['umap'] = umap_model.transform(X_test_normalized)

    return X_train, X_test

X_train_umap, X_test_umap = umap(X_train, X_test)
X_train_umap

,FTSE,EuroStoxx50,SP500,Gold,French-2Y,French-5Y,French-10Y,French-30Y,US-2Y,US-5Y,...,CRB,gold_r__div__teny_r,US-2Y_equity_interaction,US-5Y_equity_interaction,US-10Y_equity_interaction,US-30Y_equity_interaction,FTSE_FTSE_Small_Difference,pca,svd,umap
Dates,,,,,,,,,,,,,,,,,,,,,
1989-02-01,2039.70,875.470,297.09,392.50,99.081,99.039,99.572,100.000,100.031,100.345,...,286.67,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-0.009470,-0.009470,8.718415
1989-02-02,2043.40,878.080,296.84,392.00,98.898,99.117,99.278,99.692,100.000,100.314,...,287.03,2.043879,2.607829e-07,2.599668e-07,5.244780e-07,2.559093e-07,-0.002078,1.373297,1.373297,11.798780
1989-02-03,2069.90,884.090,296.97,388.75,98.907,99.002,99.145,99.178,99.812,100.062,...,285.63,8.724099,-8.233392e-07,-1.100170e-06,-4.161958e-07,-8.036502e-07,0.003126,2.638858,2.638858,10.915337
1989-02-06,2044.30,885.490,296.04,388.00,98.484,98.502,98.510,97.739,99.812,100.062,...,284.69,1.533094,-0.000000e+00,-0.000000e+00,3.940874e-06,7.696842e-06,-0.011012,2.087348,2.087348,13.001037
1989-02-07,2072.80,883.820,299.63,392.75,98.438,98.312,98.292,97.688,99.906,100.251,...,284.21,3.525563,1.142061e-05,2.290534e-05,4.210924e-05,7.457608e-05,0.010380,0.183672,0.183672,3.132634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-25,7592.66,3553.389,2683.34,1274.29,91.846,149.476,209.557,293.433,124.228,147.302,...,431.77,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-0.009163,-0.009163,8.687807
2017-12-26,7592.66,3553.389,2680.50,1282.51,91.846,149.476,209.557,293.433,124.228,147.418,...,432.65,3.376767,-0.000000e+00,-8.334739e-07,-2.021832e-06,-5.894391e-06,0.000000,-0.444257,-0.444257,1.404907
2017-12-27,7620.68,3550.174,2682.62,1286.08,91.793,149.605,209.935,294.529,124.282,147.651,...,431.77,0.582739,3.437909e-07,1.250044e-06,3.777925e-06,1.063392e-05,-0.000793,-0.914195,-0.914195,2.128363


## Extracting

In [31]:
# Extracting
# Name: Annualized volatility in returns
# Description: We are developing an annualized volatility measure for all asset returns, which is a good measure of market turbulence

def vola(df):
    volatility = df[raw_price_cols].pct_change().rolling(window=252).std()*(252**0.5)
    new_names = [(i,i+'_vol') for i in df[raw_price_cols].columns.values]
    volatility.rename(columns = dict(new_names), inplace=True)
    df = pd.concat((df, volatility), axis=1)
    return df

X_train = vola(X_train); X_test = vola(X_test)
X_train

,FTSE,EuroStoxx50,SP500,Gold,French-2Y,French-5Y,French-10Y,French-30Y,US-2Y,US-5Y,...,French-30Y_vol,US-2Y_vol,US-5Y_vol,US-10Y_vol,US-30Y_vol,Russel2000_vol,EuroStox_Small_vol,FTSE_Small_vol,MSCI_EM_vol,CRB_vol
Dates,,,,,,,,,,,,,,,,,,,,,
1989-02-01,2039.70,875.470,297.09,392.50,99.081,99.039,99.572,100.000,100.031,100.345,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989-02-02,2043.40,878.080,296.84,392.00,98.898,99.117,99.278,99.692,100.000,100.314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989-02-03,2069.90,884.090,296.97,388.75,98.907,99.002,99.145,99.178,99.812,100.062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989-02-06,2044.30,885.490,296.04,388.00,98.484,98.502,98.510,97.739,99.812,100.062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989-02-07,2072.80,883.820,299.63,392.75,98.438,98.312,98.292,97.688,99.906,100.251,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-25,7592.66,3553.389,2683.34,1274.29,91.846,149.476,209.557,293.433,124.228,147.302,...,0.119300,0.006823,0.024931,0.047688,0.104449,0.117539,0.094075,0.053706,0.092594,0.051097
2017-12-26,7592.66,3553.389,2680.50,1282.51,91.846,149.476,209.557,293.433,124.228,147.418,...,0.119174,0.006784,0.024796,0.047532,0.104266,0.117475,0.094070,0.053474,0.092634,0.050445
2017-12-27,7620.68,3550.174,2682.62,1286.08,91.793,149.605,209.935,294.529,124.282,147.651,...,0.119220,0.006775,0.024775,0.047626,0.104879,0.117224,0.093815,0.053356,0.092630,0.050293


In [32]:
## Extracting 1 
# Name: 20-Day Simple Moving Average
# Description: Calculate the 50-day SMA for gold, providing a smoothed trend over a longer period.

def sma(df):
    window=20
    column='Gold'
    df[f'{column}_SMA'] = df[column].rolling(window=window).mean()
    return df

X_train = sma(X_train); X_test = sma(X_test)
X_train

,FTSE,EuroStoxx50,SP500,Gold,French-2Y,French-5Y,French-10Y,French-30Y,US-2Y,US-5Y,...,US-2Y_vol,US-5Y_vol,US-10Y_vol,US-30Y_vol,Russel2000_vol,EuroStox_Small_vol,FTSE_Small_vol,MSCI_EM_vol,CRB_vol,Gold_SMA
Dates,,,,,,,,,,,,,,,,,,,,,
1989-02-01,2039.70,875.470,297.09,392.50,99.081,99.039,99.572,100.000,100.031,100.345,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989-02-02,2043.40,878.080,296.84,392.00,98.898,99.117,99.278,99.692,100.000,100.314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989-02-03,2069.90,884.090,296.97,388.75,98.907,99.002,99.145,99.178,99.812,100.062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989-02-06,2044.30,885.490,296.04,388.00,98.484,98.502,98.510,97.739,99.812,100.062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989-02-07,2072.80,883.820,299.63,392.75,98.438,98.312,98.292,97.688,99.906,100.251,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-25,7592.66,3553.389,2683.34,1274.29,91.846,149.476,209.557,293.433,124.228,147.302,...,0.006823,0.024931,0.047688,0.104449,0.117539,0.094075,0.053706,0.092594,0.051097,1264.3915
2017-12-26,7592.66,3553.389,2680.50,1282.51,91.846,149.476,209.557,293.433,124.228,147.418,...,0.006784,0.024796,0.047532,0.104266,0.117475,0.094070,0.053474,0.092634,0.050445,1263.7995
2017-12-27,7620.68,3550.174,2682.62,1286.08,91.793,149.605,209.935,294.529,124.282,147.651,...,0.006775,0.024775,0.047626,0.104879,0.117224,0.093815,0.053356,0.092630,0.050293,1263.8975


In [33]:
## Extracting 2
# Name: 12-Day Price Oscillator for Eurozone stocks
# Description: Calculate the difference between the 12-day and 26-day exponential moving averages for the Eurozone stocks.

def price_oscillator(df):
    short_window = 12
    long_window = 26
    column="EuroStoxx50"
    df[f'{column}_Price_Oscillator'] = df[column].ewm(span=short_window).mean() - df[column].ewm(span=long_window).mean()
    return df

X_train = price_oscillator(X_train); X_test = price_oscillator(X_test)
X_train

,FTSE,EuroStoxx50,SP500,Gold,French-2Y,French-5Y,French-10Y,French-30Y,US-2Y,US-5Y,...,US-5Y_vol,US-10Y_vol,US-30Y_vol,Russel2000_vol,EuroStox_Small_vol,FTSE_Small_vol,MSCI_EM_vol,CRB_vol,Gold_SMA,EuroStoxx50_Price_Oscillator
Dates,,,,,,,,,,,,,,,,,,,,,
1989-02-01,2039.70,875.470,297.09,392.50,99.081,99.039,99.572,100.000,100.031,100.345,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1989-02-02,2043.40,878.080,296.84,392.00,98.898,99.117,99.278,99.692,100.000,100.314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.058558
1989-02-03,2069.90,884.090,296.97,388.75,98.907,99.002,99.145,99.178,99.812,100.062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260984
1989-02-06,2044.30,885.490,296.04,388.00,98.484,98.502,98.510,97.739,99.812,100.062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.397202
1989-02-07,2072.80,883.820,299.63,392.75,98.438,98.312,98.292,97.688,99.906,100.251,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.396405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-25,7592.66,3553.389,2683.34,1274.29,91.846,149.476,209.557,293.433,124.228,147.302,...,0.024931,0.047688,0.104449,0.117539,0.094075,0.053706,0.092594,0.051097,1264.3915,-6.619936
2017-12-26,7592.66,3553.389,2680.50,1282.51,91.846,149.476,209.557,293.433,124.228,147.418,...,0.024796,0.047532,0.104266,0.117475,0.094070,0.053474,0.092634,0.050445,1263.7995,-7.346905
2017-12-27,7620.68,3550.174,2682.62,1286.08,91.793,149.605,209.935,294.529,124.282,147.651,...,0.024775,0.047626,0.104879,0.117224,0.093815,0.053356,0.092630,0.050293,1263.8975,-8.089211


In [34]:
## Extracting 3 
# Name: 14-Day RSI for Russell 2000
''' Description: The 14-day RSI measures the speed and change of price movements, providing insights into 
potential overbought or oversold conditions in the Russell 2000 index. This technical indicator aids in 
identifying trends and potential reversal points in the small-cap equity market.
'''
def rsi(df):
    window = 14
    df['Russel2000_RSI'] = 100 - (100 / (1 + df['Russel2000'].diff(1).rolling(window=window, min_periods=1).apply(lambda x: x[x > 0].mean()) / -df['SP500'].diff(1).rolling(window=window, min_periods=1).apply(lambda x: x[x < 0].mean())))
    return df

X_train = rsi(X_train); X_test = rsi(X_test)
X_train

,FTSE,EuroStoxx50,SP500,Gold,French-2Y,French-5Y,French-10Y,French-30Y,US-2Y,US-5Y,...,US-10Y_vol,US-30Y_vol,Russel2000_vol,EuroStox_Small_vol,FTSE_Small_vol,MSCI_EM_vol,CRB_vol,Gold_SMA,EuroStoxx50_Price_Oscillator,Russel2000_RSI
Dates,,,,,,,,,,,,,,,,,,,,,
1989-02-01,2039.70,875.470,297.09,392.50,99.081,99.039,99.572,100.000,100.031,100.345,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
1989-02-02,2043.40,878.080,296.84,392.00,98.898,99.117,99.278,99.692,100.000,100.314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.058558,69.135802
1989-02-03,2069.90,884.090,296.97,388.75,98.907,99.002,99.145,99.178,99.812,100.062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260984,72.375691
1989-02-06,2044.30,885.490,296.04,388.00,98.484,98.502,98.510,97.739,99.812,100.062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.397202,52.610442
1989-02-07,2072.80,883.820,299.63,392.75,98.438,98.312,98.292,97.688,99.906,100.251,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.396405,59.216590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-25,7592.66,3553.389,2683.34,1274.29,91.846,149.476,209.557,293.433,124.228,147.302,...,0.047688,0.104449,0.117539,0.094075,0.053706,0.092594,0.051097,1264.3915,-6.619936,71.967101
2017-12-26,7592.66,3553.389,2680.50,1282.51,91.846,149.476,209.557,293.433,124.228,147.418,...,0.047532,0.104266,0.117475,0.094070,0.053474,0.092634,0.050445,1263.7995,-7.346905,67.444097
2017-12-27,7620.68,3550.174,2682.62,1286.08,91.793,149.605,209.935,294.529,124.282,147.651,...,0.047626,0.104879,0.117224,0.093815,0.053356,0.092630,0.050293,1263.8975,-8.089211,66.674871
